# Eliptic PDE-s

## Laplace equation

Let us start by discretizing the stationary heat equation in a rectangular plated with dimension as given in [Figure](#fig:73):

<!-- Equation labels as ordinary links -->
<div id="eq:7201"></div>

$$
\begin{equation}
  \frac{\partial^2 T}{\partial x^2} + \frac{\partial^2 T}{\partial
    y^2}=0
\label{eq:7201} \tag{1}
\end{equation}
$$

<!-- dom:FIGURE:[fig/4.png, width=400 frac=0.6] Rectangular domain with prescribed values at the boundaries (Dirichlet). <div id="fig:73"></div> -->
<!-- begin figure -->
<div id="fig:73"></div>

<p>Rectangular domain with prescribed values at the boundaries (Dirichlet).</p>
<img src="fig/4.png" width=400>

<!-- end figure -->



We adopt the following notation:

$$
x_i=x_0+i\cdot h,\ i=0,1,2,\dots
$$

$$
y_j=y_0+j\cdot h,\ j=0,1,2,\dots
$$

For convenience we assume $\Delta x = \Delta y = h$. The ordering of
the unkown temperatures is illsustrated in .

<!-- dom:FIGURE:[fig/3_new_transparent.png, width=400 frac=0.6] Illustration of the numerical stencil. <div id="fig:ch7-3"></div> -->
<!-- begin figure -->
<div id="fig:ch7-3"></div>

<p>Illustration of the numerical stencil.</p>
<img src="fig/3_new_transparent.png" width=400>

<!-- end figure -->


By approximation the second order differentials in [(1)](#eq:7201) by central differences we get the following numerical stencil:

<!-- Equation labels as ordinary links -->
<div id="eq:7202"></div>

$$
\begin{equation}
  T_{i+1,j}+T_{i-1,j}+T_{i,j+1}+T_{i,j-1}-4 T_{i,j}=0
\label{eq:7202} \tag{2}
\end{equation}
$$

which states that the temperature $T_{i,j}$ in at the location $(i,j)$ depends on the values of its neighbors to the left, right, up and down. Frequently, the neighbors are denoted in compass notation, i.e. $\text{west}=i-1$, $\text{east}=i+1$, $\text{south}=j-1$, and $\text{north}=j+1$. By referring to the compass directions with their first letters, and equivalent representation of the stencil in [(2)](#eq:7202) reads:

<!-- Equation labels as ordinary links -->
<div id="eq:compass"></div>

$$
\begin{equation}
  T_{e}+T_{w}+T_{n}+T_{s}-4 T_{m}=0
  \label{eq:compass} \tag{3}
\end{equation}
$$

<!-- dom:FIGURE:[fig/3_compass_sanseriff.png, width=400 frac=0.6] Illustration of the numerical stencil with compass notation. <div id="fig:ch7-3_compass"></div> -->
<!-- begin figure -->
<div id="fig:ch7-3_compass"></div>

<p>Illustration of the numerical stencil with compass notation.</p>
<img src="fig/3_compass_sanseriff.png" width=400>

<!-- end figure -->


The smoothing nature of elliptic problems may be seen even more clearly by isolating the $T_{i,j}$ in [(3)](#eq:compass) on the left hand side:

<!-- Equation labels as ordinary links -->
<div id="eq:avearge"></div>

$$
\begin{equation}
T_{m}= \frac{T_{e}+T_{w}+T_{n}+T_{s}}{4}
\label{eq:avearge} \tag{4}
\end{equation}
$$

showing that the temperature $T_{m}$ in each point is the average temperature
of the neighbors (to the east, west, north, and south).


The temperature is prescribed at the bondaries (i.e. Dirichlet
boundary conditions) and are given by:

$$
\begin{equation}
  T=0.0 \quad \textrm{at} \quad  y=0 \nonumber 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:BCs"></div>

$$
\begin{equation} 
  T=0.0 \quad \textrm{at} \quad  x=0  \quad \textrm{ and }\quad  x=1 \quad  \textrm{ for } \quad 0 \leq y < 1.5 \label{eq:BCs} \tag{5}
\end{equation}
$$

$$
\begin{equation} 
  T=100.0 \quad \textrm{at} \quad y=1.5 \nonumber
\end{equation}
$$

In [1]:
import sympy as sp
sp.init_printing()

In [2]:
def indice(i, j):
    return (i, j)

nx = 3
ny = 5 
sp.Matrix(ny + 2, nx + 2, indice)

In [3]:
def TPoints(i, j):
    n = j*ny + (ny-i)
    expr = 'T' + str(n)
    expr = sp.symbols(expr)
    return expr
    
Points = sp.Matrix(ny, nx, TPoints)
Points

In [4]:
def TPointsFull(i, j):
    if i == 0:
        point = 100
    elif i == ny + 1:
        point = 0
    elif j == 0:
        point = 0
    elif j == nx + 1:
        point = 0
    else:
        n = (j-1)*ny + (ny - i + 1)
        expr = 'T' + str(n)
        point = sp.symbols(expr)
    return point
    
    
def unknown(i, j):
    expr = 'T' + str(i + 1)
    expr = sp.symbols(expr)
    return expr


T = sp.Matrix(ny + 2, nx + 2, TPointsFull)

In [5]:
def differenceEqs(n):
    j = (n-1)/ny + 1# integer division
    n = n -(j-1)*ny
    
    i = ny - n + 1
    diffeq = T[i-1,j] + T[i+1,j] + T[i,j-1] + T[i,j+1] - 4*T[i,j]
    print "i={0}, j={1}".format(i, j)
    return diffeq
    
differenceEqs(1)

In [6]:
Eqs = []
for n in range(nx*ny):
    Eqs.append([differenceEqs(n + 1)])
sp.Matrix(Eqs)

In [7]:
import numpy as np
import scipy 
import scipy.sparse
import scipy.sparse.linalg

nx = nx*2
ny = ny*2
N = nx*ny
mainD = -4*np.ones(N) # main diagonal

d = np.zeros(N) # RHS
d[ny-1::ny] = -100 # every ny element on RHS is -100
superD = np.ones(N - 1)
subD = np.ones(N - 1)

superD[ny-1::ny] = 0 # every ny element on first diagonal is zero
subD[ny-1::ny] = 0

superD5 = np.ones(N - ny)
subD5 = np.ones(N - ny)

#A = scipy.sparse.diags([subD5, subD, mainD, superD, superD5], [-ny, -1, 0, 1, ny], format='csc', dtype=int) # should remove..
# dtype=int when solving
#sp.Matrix(A.toarray()), sp.Matrix(N, 1, unknown), sp.Matrix(d)

In [8]:
def TPointsFill(i, j):
    if i == 0:
        point = 100
    elif i == ny + 1:
        point = 0
    elif j == 0:
        point = 0
    elif j == nx + 1:
        point = 0
    else:
        n = (j-1)*ny + (ny - i + 1)
        point = round(T[n - 1], 2)
    return point

A = scipy.sparse.diags([subD5, subD, mainD, superD, superD5], [-ny, -1, 0, 1, ny], format='csc')
T = scipy.sparse.linalg.spsolve(A, d)
sp.Matrix(ny + 2, nx + 2, TPointsFill)

In [9]:
%matplotlib inline

%matplotlib inline
import matplotlib.pylab as plt
from Visualization import plot_Surface_yx
fig = plot_Surface_yx(T, 100, 1, 1.5, nx + 1, ny + 1, nx, ny)
plt.show()